In [ ]:
# 源码

In [ ]:
# %load train_kronnc.py
# random del 15 个nodes  email.txt
import torch
import pickle
import random
import numpy as np
import networkx as nx 
from networkx.convert import from_dict_of_dicts
from networkx.classes.graph import Graph
# from kronEM import *
seed =1900
np.random.seed(seed)
random.seed(seed)

In [2]:
# generate partial obseved data

import torch 
import torch.nn as nn
from torch.nn.parameter import Parameter 
import torch.optim as optim
# set seed 
seed = 1322
torch.manual_seed(seed)

In [3]:
def kronecker(A,B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))
def partial_mask(sz,del_num):
    '''mask the known part and un know part'''
    kn_mask = torch.zeros(sz,sz)
    kn_mask[:-del_num,:-del_num] = 1
    
    un_un_mask = torch.zeros(sz,sz)
    un_un_mask[-del_num:,-del_num:] = 1
    un_un_mask = un_un_mask - torch.diag(torch.diag(un_un_mask))
    
    left_mask = torch.ones(sz,sz)
    left_mask[:-del_num,:-del_num] = 0
    left_mask = left_mask-torch.diag(torch.diag(left_mask))
    kn_un_mask = left_mask - un_un_mask
    
    return kn_mask,left_mask,un_un_mask,kn_un_mask


class kronecker_Generator(nn.Module):
    def __init__(self,korder = 3,node_num = 2):
        super(kronecker_Generator,self).__init__()
        self.p = Parameter(torch.rand(node_num,node_num,requires_grad=True))
        self.korder = korder
        # print(self.p)
    def generator_adjacency(self):
        k = self.korder
        p0 = self.p
        adj = self.p
        for i in range(k-1):
            adj = kronecker(adj,p0)
        return adj

In [4]:
# toy train data  generator 
p = torch.FloatTensor([[0.9,0.7],[0.5, 0.3]])
k = 4
ground_adj = p

# 删除0.25比例的节点 origin: we randomly removed 25% of the nodes and the corresponding edges.
sz = 2**4
remove_proportion = 0.25
del_num = int(sz*remove_proportion)

for i in range(k-1):
    ground_adj = kronecker(ground_adj,p)

Ground_truth_adj = (ground_adj>torch.rand((ground_adj.shape))).float()
sigma=Ground_truth_adj

kn_mask,left_mask,un_un_mask,kn_un_mask = partial_mask(sz,del_num)
known_adj =  kn_mask*Ground_truth_adj

missing_edges = int((left_mask*Ground_truth_adj).sum())

# initial partial z with fixed missing edges num
z_element_num = int(left_mask.sum())
z_element_shuffle = torch.randperm(z_element_num)
z_element_choice = z_element_shuffle[:missing_edges]
z_element_left = z_element_shuffle[missing_edges:] 
z_element = torch.nonzero(left_mask)
init_z_edges = torch.index_select(z_element,0,z_element_choice)
init_z_nonedges = torch.index_select(z_element,0,z_element_left)
init_z_adj = torch.zeros([16,16])
for (i,j) in init_z_edges:
  init_z_adj[i][j] = 1

In [5]:
generator = kronecker_Generator(4,2)
Pk = generator.generator_adjacency()

In [ ]:
RunEStep()
RunEStep(GibbsWarmUp, WarmUp, NSamples, LLV, DLLV)



In [ ]:
# missing edges 
# main 函数

MissEdges = NMissing;


// KronEM
void TKroneckerLL::RunKronEM(const int& EMIter, const int& GradIter, double LrnRate, double MnStep, double MxStep, const int& GibbsWarmUp, const int& WarmUp, const int& NSamples, const TKronEMType& Type, const int& NMissing) {
	printf("\n----------------------------------------------------------------------\n");
	printf("Fitting graph on %d nodes, %d edges\n", int(RealNodes), int(RealEdges));
	printf("Kron iters:  %d (== %d nodes)\n\n", KronIters(), ProbMtx.GetNodes(KronIters()));

	TFltV LLV(NSamples);
	TVec<TFltV> DLLV(NSamples);
	//int count = 0;

	EMType = Type;
	MissEdges = NMissing;
	AppendIsoNodes();
    
    
	SetRndPerm();
	for(int i = 0; i < EMIter; i++) {
		printf("\n----------------------------------------------------------------------\n");
		printf("%03d EM-iter] E-Step\n", i+1);
		RunEStep(GibbsWarmUp, WarmUp, NSamples, LLV, DLLV);
		printf("\n\n");

		printf("%03d EM-iter] M-Step\n", i+1);
		double CurLL = RunMStep(LLV, DLLV, GradIter, LrnRate, MnStep, MxStep);
		printf("\n\n");

		if(DebugMode) {
			LLV.Add(CurLL);
			MtxV.Add(ProbMtx);
		}
	}

	RestoreGraph();
}